In [10]:
import pandas as pd
import plotly.express as px
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from scipy.stats import poisson 
import math
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from sklearn import preprocessing

def sum_of_lower_upper(mat):
    r = len(mat)
    c = len(mat[0])
    diag = np.trace(mat)
    lower_sum = 0
    upper_sum = 0
    over_sum = 0
    under_sum = 0

    for i in range(r):
        for j in range(c):
            if (i<j):
                upper_sum += mat[i][j]

    for i in range(r):
        for j in range(c):
            if (j<i):
                lower_sum += mat[i][j]
    
    for i in range(r):
        for j in range(c):
            if (i+j>=3):
                over_sum += mat[i][j]
            else:
                under_sum += mat[i][j]

    return (upper_sum, lower_sum, diag, over_sum, under_sum)


## Gathering Lamda from season long performance

In [11]:
df1 = pd.read_csv('GatheredData/Full20xxFiles/Full2017.csv')
df2 = pd.read_csv('GatheredData/Full20xxFiles/Full2018.csv')
df3 = pd.read_csv('GatheredData/Full20xxFiles/Full2019.csv')
df4 = pd.read_csv('GatheredData/Full20xxFiles/Full2020.csv')
df5 = pd.read_csv('GatheredData/Full20xxFiles/Full2021.csv')
df6 = pd.read_csv('GatheredData/Full20xxFiles/Full2022.csv')
PRED = pd.read_csv('Prediction.csv')
pred = PRED.copy()
#Keeping bookmakers odds
Odds = PRED.copy()
Odds = PRED[['HTeam', 'ATeam','B365H', 'B365D', 'B365A', 'BbAv>2.5', 'BbAv<2.5']]

df1 = df1[['Date_x', 'HxG', 'HM', 'HTeam', 'ATeam', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'Result', 'B365H', 'B365D', 'B365A', 'BbAv>2.5', 'BbAv<2.5']]
df2 = df2[['Date_x', 'HxG', 'HM', 'HTeam', 'ATeam', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'Result', 'B365H', 'B365D', 'B365A', 'BbAv>2.5', 'BbAv<2.5']]
df3 = df3[['Date_x', 'HxG', 'HM', 'HTeam', 'ATeam', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'Result', 'B365H', 'B365D', 'B365A', 'AvgC>2.5', 'AvgC<2.5']]
df3.rename(columns = {'AvgC<2.5':'BbAv<2.5'}, inplace = True)
df3.rename(columns = {'AvgC>2.5':'BbAv>2.5'}, inplace = True)
df4 = df4[['Date_x', 'HxG', 'HM', 'HTeam', 'ATeam', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'Result', 'B365H', 'B365D', 'B365A', 'AvgC>2.5', 'AvgC<2.5']]
df4.rename(columns = {'AvgC<2.5':'BbAv<2.5'}, inplace = True)
df4.rename(columns = {'AvgC>2.5':'BbAv>2.5'}, inplace = True)
df5 = df5[['Date_x', 'HxG', 'HM', 'HTeam', 'ATeam', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'Result', 'B365H', 'B365D', 'B365A', 'AvgC>2.5', 'AvgC<2.5']]
df5.rename(columns = {'AvgC<2.5':'BbAv<2.5'}, inplace = True)
df5.rename(columns = {'AvgC>2.5':'BbAv>2.5'}, inplace = True)
df6 = df6[['Date', 'HxG', 'HM', 'HTeam', 'ATeam', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'Result', 'B365H', 'B365D', 'B365A', 'B365>2.5', 'B365<2.5']]
df6.rename(columns = {'B365<2.5':'BbAv<2.5'}, inplace = True)
df6.rename(columns = {'B365>2.5':'BbAv>2.5'}, inplace = True)

df1.rename(columns = {'Date_x':'Date'}, inplace = True)
df2.rename(columns = {'Date_x':'Date'}, inplace = True)
df3.rename(columns = {'Date_x':'Date'}, inplace = True)
df4.rename(columns = {'Date_x':'Date'}, inplace = True)
df5.rename(columns = {'Date_x':'Date'}, inplace = True)

Season = pd.read_csv('GatheredData/Full20xxFiles/Full2023.csv')
Season = Season[['HxG', 'HM', 'HTeam', 'ATeam', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'Result', 'B365H', 'B365D', 'B365A', 'BbAv>2.5', 'BbAv<2.5']]
Season = Season.iloc[38:]

DF = pd.concat([df1, df2, df3, df4, df5, df6, Season], ignore_index=True)


DF = DF[ DF['HM'] != 0] 
DF = DF[ DF['HM'] != 1] 
DF = DF[ DF['HM'] != 2] 
DF = DF[ DF['HM'] != 3]
subset = DF[['Date', 'HTeam', 'ATeam']].copy()
extra_stats = DF.copy()
extra_pred = pred.copy()

DF['PTSDIF/M']=DF['HPTS/M']-DF['APTS/M']
pred['PTSDIF/M']=pred['HPTS/M']-pred['APTS/M']
DF = DF[[ 'HxG', 'HM', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'PTSDIF/M', 'Result']]
pred = pred[[ 'HxG', 'HM', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'PTSDIF/M', 'Result']]
cut = DF.shape[0]
DF = pd.concat([DF, pred], ignore_index=True)
try:
    DF['HxG'] = DF['HxG'].div(DF['HM'])
    DF['HG'] = DF['HG'].div(DF['HM'])
    DF['HGA'] = DF['HGA'].div(DF['HM'])
    DF['HxGA'] = DF['HxGA'].div(DF['HM'])
    DF['AxG'] = DF['AxG'].div(DF['AM'])
    DF['AG'] = DF['AG'].div(DF['AM'])
    DF['AxGA'] = DF['AxG'].div(DF['AM'])
    DF['AGA'] = DF['AGA'].div(DF['AM'])
    pred['HxG'] = pred['HxG'].div(pred['HM'])
    pred['HG'] = pred['HG'].div(pred['HM'])
    pred['HGA'] = pred['HGA'].div(pred['HM'])
    pred['HxGA'] = pred['HxGA'].div(pred['HM'])
    pred['AxG'] = pred['AxG'].div(pred['AM'])
    pred['AG'] = pred['AG'].div(pred['AM'])
    pred['AxGA'] = pred['AxG'].div(pred['AM'])
    pred['AGA'] = pred['AGA'].div(pred['AM'])

except ZeroDivisionError:
    DF['HxG'] = 0
    DF['HG'] = 0
    DF['HGA'] = 0
    DF['HxGA'] = 0
    DF['AxG'] = 0
    DF['AG'] = 0
    DF['AxGA'] = 0
    DF['AGA'] = 0

HGoals = []
AGoals = []
    
for i, x in DF.iloc[:cut].iterrows():
    a=x['Result'].split('\'')
    HGoals.append(a[1])
    AGoals.append(a[3])
    
DF=np.array(DF)
DFnorm = preprocessing.normalize(DF[:, 0:21], axis=0)
pred = DFnorm[cut:]
DF = DFnorm[:cut]

x=DF


y1=np.array(HGoals)
y2=np.array(AGoals)

reg1 = LinearRegression().fit(x, y1)
reg2 = LinearRegression().fit(x, y2)

LamdaHome= reg1.predict(pred)
LamdaAway= reg2.predict(pred)

## Gathering Lamda from 2 month form performance

In [14]:
df1 = pd.read_csv('GatheredData/Epl20xx_3Months/Epl2017_3Months.csv')
df2 = pd.read_csv('GatheredData/Epl20xx_3Months/Epl2018_3Months.csv')
df3 = pd.read_csv('GatheredData/Epl20xx_3Months/Epl2019_3Months.csv')
df4 = pd.read_csv('GatheredData/Epl20xx_3Months/Epl2020_3Months.csv')
df5 = pd.read_csv('GatheredData/Epl20xx_3Months/Epl2021_3Months.csv')
df6 = pd.read_csv('GatheredData/Epl20xx_3Months/Epl2022_3Months.csv')
PRED = pd.read_csv('PredictionMonth3.csv')
pred = PRED.copy()
#Keeping bookmakers odds
Odds = PRED.copy()
Odds = PRED[['HTeam', 'ATeam','B365H', 'B365D', 'B365A', 'BbAv>2.5', 'BbAv<2.5']]

df1 = df1[['Date', 'HxG', 'HM', 'HTeam', 'ATeam', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'Result', 'B365H', 'B365D', 'B365A', 'BbAv>2.5', 'BbAv<2.5']]
df2 = df2[['Date', 'HxG', 'HM', 'HTeam', 'ATeam', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'Result', 'B365H', 'B365D', 'B365A', 'BbAv>2.5', 'BbAv<2.5']]
df3 = df3[['Date', 'HxG', 'HM', 'HTeam', 'ATeam', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'Result', 'B365H', 'B365D', 'B365A', 'BbAv>2.5', 'BbAv<2.5']]
df4 = df4[['Date', 'HxG', 'HM', 'HTeam', 'ATeam', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'Result', 'B365H', 'B365D', 'B365A', 'BbAv>2.5', 'BbAv<2.5']]
df5 = df5[['Date', 'HxG', 'HM', 'HTeam', 'ATeam', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'Result', 'B365H', 'B365D', 'B365A', 'BbAv>2.5', 'BbAv<2.5']]
df6 = df6[['Date', 'HxG', 'HM', 'HTeam', 'ATeam', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'Result', 'B365H', 'B365D', 'B365A', 'B365>2.5', 'B365<2.5']]
df6.rename(columns = {'B365<2.5':'BbAv<2.5'}, inplace = True)
df6.rename(columns = {'B365>2.5':'BbAv>2.5'}, inplace = True)

SeasonForm = pd.read_csv('GatheredData/Epl20xx_3Months/Epl2023_3Months.csv')
    
SeasonForm = SeasonForm[['HxG', 'HM', 'HTeam', 'ATeam', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'Result', 'B365H', 'B365D', 'B365A', 'BbAv>2.5', 'BbAv<2.5']]
SeasonForm = SeasonForm.iloc[38:]

DF = pd.concat([df1, df2, df3, df4, df5, df6, SeasonForm], ignore_index=True)
merged = pd.merge(subset, DF, on=['Date', 'HTeam', 'ATeam'], how='left')
# resulting dataframe will have columns of B where the triplet of A['a', 'b', 'c'] exists in B
DF = merged[DF.columns]
DF['PTSDIF/M']=DF['HPTS/M']-DF['APTS/M']
pred['PTSDIF/M']=pred['HPTS/M']-pred['APTS/M']
DF = DF[['HxG', 'HM', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'PTSDIF/M', 'Result']]
pred = pred[[ 'HxG', 'HM', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'PTSDIF/M', 'Result']]
cut = DF.shape[0]
DF = pd.concat([DF, pred], ignore_index=True)


try:
    DF['HxG'] = DF['HxG'].div(DF['HM']).fillna(0)
    DF['HG'] = DF['HG'].div(DF['HM']).fillna(0)
    DF['HGA'] = DF['HGA'].div(DF['HM']).fillna(0)
    DF['HxGA'] = DF['HxGA'].div(DF['HM']).fillna(0)
    DF['AxG'] = DF['AxG'].div(DF['AM']).fillna(0)
    DF['AG'] = DF['AG'].div(DF['AM']).fillna(0)
    DF['AxGA'] = DF['AxG'].div(DF['AM']).fillna(0)
    DF['AGA'] = DF['AGA'].div(DF['AM']).fillna(0)
    pred['HxG'] = pred['HxG'].div(pred['HM']).fillna(0)
    pred['HG'] = pred['HG'].div(pred['HM']).fillna(0)
    pred['HGA'] = pred['HGA'].div(pred['HM']).fillna(0)
    pred['HxGA'] = pred['HxGA'].div(pred['HM']).fillna(0)
    pred['AxG'] = pred['AxG'].div(pred['AM']).fillna(0)
    pred['AG'] = pred['AG'].div(pred['AM']).fillna(0)
    pred['AxGA'] = pred['AxG'].div(pred['AM']).fillna(0)
    pred['AGA'] = pred['AGA'].div(pred['AM']).fillna(0)

except ZeroDivisionError:
    DF['HxG'] = 0
    DF['HG'] = 0
    DF['HGA'] = 0
    DF['HxGA'] = 0
    DF['AxG'] = 0
    DF['AG'] = 0
    DF['AxGA'] = 0
    DF['AGA'] = 0

HGoals = []
AGoals = []
for i, x in DF.iloc[:cut].iterrows():
    a=x['Result'].split('\'')
    HGoals.append(a[1])
    AGoals.append(a[3])

DF=np.array(DF)

DFnorm = preprocessing.normalize(DF[:, 0:21].astype('float64'), axis=0)
pred = DFnorm[cut:]
DF = DFnorm[:cut]

x=DF

y1=np.array(HGoals)
y2=np.array(AGoals)

reg1 = LinearRegression().fit(x, y1)
reg2 = LinearRegression().fit(x, y2)

LamdaHomeForm= reg1.predict(pred)
LamdaAwayForm= reg2.predict(pred)

FileNotFoundError: [Errno 2] No such file or directory: 'GatheredData/Epl-20xx_3Months/Epl-2022_3Months.csv'

In [ ]:
LamdaAwayFinal = [(LamdaAway[i] * 0.4 + LamdaAwayForm[i] * 0.6) for i in range(len(LamdaAway))]
LamdaHomeFinal = [(LamdaHome[i] * 0.4 + LamdaHomeForm[i] * 0.6) for i in range(len(LamdaHome))]

print(LamdaHomeFinal)
print(LamdaAwayFinal)

[1.5337805627687664, 2.695917949896608, 1.6248958148757129, 1.1573753729307383, 2.2785026284805427, 1.5100224274366214, 1.2303069035921292, 1.2177294662486253, 2.356414527446355, 1.3949932757946897, 1.0322637129155061, 1.5089366077656017, 1.5220024646701327, 1.3102737701703804, 1.8624398691076347, 2.076780029624154, 0.6362522075943216, 1.4921185257650358, 2.191746595545063, 0.8933847170443479]
[1.5239425367385522, 0.5009478890427725, 0.9451263210032039, 1.3455541770035557, 0.8983916327355536, 1.6936791915344558, 1.6362955905093615, 1.3273543438041808, 0.7518767374912625, 1.280424363837858, 1.733439207319727, 1.1725741868125135, 1.3050356790211706, 1.2520323159091218, 0.8028048501259439, 0.956883807173392, 2.2049411203385443, 1.0576724096285681, 1.4650639149917384, 1.8667116023555306]


In [ ]:
k = range(12)
PH = np.zeros((len(k), 1))
PA = np.zeros((1, len(k)))
PRED['P(Hwin)']= np.nan
PRED['P(Awin)']= np.nan
PRED['P(Draw)']= np.nan
PRED['P(Over)']= np.nan
PRED['P(Under)']= np.nan

Pie_list = []
for j, l in enumerate(LamdaHome):
    for i in k:
        PH[i, 0] = (math.exp(-l)*l**k[i]) /math.factorial(k[i])
        PA[0, i] = (math.exp(-LamdaAway[j])*LamdaAway[j]**k[i]) /math.factorial(k[i])
    Pscore = np.matmul(PH, PA) 
    df = pd.DataFrame(Pscore, columns = ['0','1','2', '3', '4', '5', '6', '7', '8', '9', '10', '11'])
    PRED.loc[j,'P(Awin)'], PRED.loc[j,'P(Hwin)'], PRED.loc[j,'P(Draw)'], PRED.loc[j,'P(Over)'], PRED.loc[j,'P(Under)']  = sum_of_lower_upper(Pscore)
    PRED['P(Over)'] = 1 - PRED['P(Under)']
    df = df.iloc[0:5, 0:5]
    pd.set_option('display.float_format', lambda x: '%.3f' % x)
    
    G = []
    for i in range(0, 5):
        for j in range(0, 5):
            G.append(df.loc[i][j])

            
    Pie_Df = pd.DataFrame(G, columns = ['P(score)'])
    Pie_Df['score']= ['0 - 0',
                      '0 - 1',
                      '0 - 2',
                      '0 - 3',
                      '0 - 4',
                      '1 - 0',
                      '1 - 1',
                      '1 - 2',
                      '1 - 3',
                      '1 - 4',
                      '2 - 0',
                      '2 - 1',
                      '2 - 2',
                      '2 - 3',
                      '2 - 4',
                      '3 - 0',
                      '3 - 1',
                      '3 - 2',
                      '3 - 3',
                      '3 - 4',
                      '4 - 0',
                      '4 - 1',
                      '4 - 2',
                      '4 - 3',
                      '4 - 4']
    Pie_Df.loc[Pie_Df["P(score)"] < 0.02, "score"] = 'Other'
    Pie_Df = Pie_Df.sort_values(by=['P(score)'], ascending=False)
    Pie_list.append(Pie_Df)
    

PRED = PRED[['HTeam', 'ATeam', 'P(Hwin)', 'P(Draw)', 'P(Awin)', 'P(Under)', 'P(Over)']]
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(PRED)

Final_Odds = pd.concat([PRED, Odds], axis=0)
Final_Odds['Type'] = np.nan
Final_Odds['Match'] = np.nan
Final_Odds = Final_Odds.reset_index(drop = 'True')
Final_Odds.loc[Final_Odds['B365H'].isnull(), 'Type'] = 'Predicted'
Final_Odds.loc[Final_Odds['P(Hwin)'].isnull(), 'Type'] = 'Bookmaker'

for i, x in enumerate(Final_Odds['Match']):
    Final_Odds['Match'][i] = str(Final_Odds['HTeam'][i] + '-' + Final_Odds['ATeam'][i])

li  = Final_Odds.index[Final_Odds['Type'] == 'Bookmaker'].tolist()
for i in li:
    Final_Odds['P(Hwin)'][i] = 1/Final_Odds.iloc[i]['B365H']
    Final_Odds['P(Awin)'][i] = 1/Final_Odds.iloc[i]['B365A']
    Final_Odds['P(Draw)'][i] = 1/Final_Odds.iloc[i]['B365D']
    Final_Odds['P(Over)'][i] = 1/Final_Odds.iloc[i]['BbAv>2.5']
    Final_Odds['P(Under)'][i] = 1/Final_Odds.iloc[i]['BbAv<2.5']

Final_Odds = Final_Odds[['P(Hwin)', 'P(Draw)', 'P(Awin)', 'P(Under)', 'P(Over)', 'Type', 'Match']].copy()

,HTeam,ATeam,P(Hwin),P(Draw),P(Awin),P(Under),P(Over)
0,Bournemouth,Chelsea,0.351,0.246,0.403,0.439,0.561
1,Manchester City,Leeds,0.847,0.110,0.044,0.358,0.642
2,Tottenham,Crystal Palace,0.605,0.222,0.172,0.485,0.515
3,Wolverhampton Wanderers,Aston Villa,0.337,0.264,0.399,0.521,0.479
4,Liverpool,Brentford,0.657,0.189,0.153,0.373,0.627
5,Newcastle United,Arsenal,0.379,0.230,0.391,0.358,0.642
6,West Ham,Manchester United,0.315,0.250,0.435,0.467,0.533
7,Fulham,Leicester,0.351,0.270,0.378,0.545,0.455
8,Brighton,Everton,0.737,0.170,0.093,0.441,0.559
9,Nottingham Forest,Southampton,0.429,0.258,0.313,0.502,0.498


/tmp/ipykernel_147206/3739398557.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Final_Odds['Match'][i] = str(Final_Odds['HTeam'][i] + '-' + Final_Odds['ATeam'][i])
/tmp/ipykernel_147206/3739398557.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Final_Odds['P(Hwin)'][i] = 1/Final_Odds.iloc[i]['B365H']
/tmp/ipykernel_147206/3739398557.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Final_Odds['P(Awin)'][i] = 1/Final_Odds.

In [ ]:
fig = make_subplots(rows=2, cols=1)
fig.update_layout(width=1500, height=1000)
fig.add_trace(
    go.Bar(name='Predicted Home Win',
           x=Final_Odds[Final_Odds['Type']=='Predicted']['Match'],
           y=Final_Odds[Final_Odds['Type']=='Predicted']['P(Hwin)'],
           marker_color='rgb(141,211,199)'),
    row=1, col=1)

fig.add_trace(
    go.Bar(name='Bookmaker Home Win',
           x=Final_Odds[Final_Odds['Type']=='Bookmaker']['Match'],
           y=Final_Odds[Final_Odds['Type']=='Bookmaker']['P(Hwin)'],
           marker_color='rgb(82, 106, 131)'),
    row=1, col=1)

fig.add_trace(
    go.Bar(name='Predicted Draw',
           x=Final_Odds[Final_Odds['Type']=='Predicted']['Match'],
           y=Final_Odds[Final_Odds['Type']=='Predicted']['P(Draw)'],
           marker_color='rgb(141,211,199)'),
    row=1, col=1)

fig.add_trace(
    go.Bar(name='Bookmaker Draw',
           x=Final_Odds[Final_Odds['Type']=='Bookmaker']['Match'],
           y=Final_Odds[Final_Odds['Type']=='Bookmaker']['P(Draw)'],
           marker_color='rgb(82, 106, 131)'),
    row=1, col=1)

fig.add_trace(
    go.Bar(name='Predicted Away Win',
           x=Final_Odds[Final_Odds['Type']=='Predicted']['Match'],
           y=Final_Odds[Final_Odds['Type']=='Predicted']['P(Awin)'],
           marker_color='rgb(141,211,199)'),
    row=1, col=1)

fig.add_trace(
    go.Bar(name='Bookmaker Away Win',
           x=Final_Odds[Final_Odds['Type']=='Bookmaker']['Match'],
           y=Final_Odds[Final_Odds['Type']=='Bookmaker']['P(Awin)'],
           marker_color='rgb(82, 106, 131)'),
    row=1, col=1)


fig.add_trace(
    go.Bar(name='Predicted Over',
           x=Final_Odds[Final_Odds['Type']=='Predicted']['Match'],
           y=Final_Odds[Final_Odds['Type']=='Predicted']['P(Over)'],
           marker_color='rgb(141,211,199)'),
    row=2, col=1)

fig.add_trace(
    go.Bar(name='Bookmaker Over',
           x=Final_Odds[Final_Odds['Type']=='Bookmaker']['Match'],
           y=Final_Odds[Final_Odds['Type']=='Bookmaker']['P(Over)'],
           marker_color='rgb(82, 106, 131)'),
    row=2, col=1)

fig.add_trace(
    go.Bar(name='Predicted Under',
           x=Final_Odds[Final_Odds['Type']=='Predicted']['Match'],
           y=Final_Odds[Final_Odds['Type']=='Predicted']['P(Under)'],
           marker_color='rgb(141,211,199)'),
    row=2, col=1)

fig.add_trace(
    go.Bar(name='Bookmaker Under',
           x=Final_Odds[Final_Odds['Type']=='Bookmaker']['Match'],
           y=Final_Odds[Final_Odds['Type']=='Bookmaker']['P(Under)'],
           marker_color='rgb(82, 106, 131)'),
    row=2, col=1)


fig.show()


In [ ]:
# Create subplots
fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'polar'}, {'type': 'xy'}]], column_widths=[0.33, 0.67])
fig.update_layout(width=1500, height=650)
# create a list to store the buttons
buttons = []
categories = ['Home', 'Draw', 'Away', 'Over', 'Under']

# loop through each match and create two traces for Bookmaker odds and predicted values
for match_index, match in enumerate(Final_Odds['Match'].unique().tolist()):

    # create the trace for Bookmaker odds
    trace_bookmaker = go.Scatterpolar(
      r=Final_Odds[(Final_Odds['Type']=='Bookmaker')  & (Final_Odds['Match']==match)][['P(Hwin)', 'P(Draw)', 'P(Awin)', 'P(Over)', 'P(Under)']].values.tolist()[0],
      theta=categories,
      fill='toself',
      name='Bookmaker Odds',
      marker = dict(color = 'rgb(82, 106, 131)' ),
      visible=False if match_index!=0 else True,
      hovertemplate='%{theta}<br>Prob: %{r:.2f}<br>Bookmaker Odds'
)
    fig.add_trace(trace_bookmaker, row=1, col=1)
    # create the trace for predicted values
    trace_predicted = go.Scatterpolar(
        r= Final_Odds[(Final_Odds['Type']=='Predicted')  & (Final_Odds['Match']==match)][['P(Hwin)', 'P(Draw)', 'P(Awin)', 'P(Over)', 'P(Under)']].values.tolist()[0],
        theta=categories,
        fill='toself',
        name='Predicted Odds',
        marker = dict(color ='rgb(141,211,199)'), #'rgb(217, 175, 107)'
        visible=False if match_index!=0 else True,
        hovertemplate='%{theta}<br>Prob: %{r:.2f}<br>Predicted Odds'
  )
    fig.add_trace(trace_predicted, row=1, col=1)
    
    trace_goals = go.Bar(
        x=Pie_list[match_index]['score'],
        y=Pie_list[match_index]['P(score)'],
        visible=False if match_index!=0 else True,
        marker=dict(color=Pie_list[match_index]['P(score)'], colorscale='darkmint'),
        name='Propable Scoreline',
        hovertemplate='Score %{x}<br>Prob: %{y:.3f}'
    )
    fig.add_trace(trace_goals, row=1, col=2)
    
    # create the dictionary for the button
    button_dict = dict(
        method="restyle",
        args=[{"visible": [False] * (len(Final_Odds['Match'].unique().tolist()) * 3)}], # set all traces to false initially
        label=match
    )
    
    # set visibility to true for the corresponding traces
    button_dict["args"][0]["visible"][3*match_index] = True # Bookmaker odds trace
    button_dict["args"][0]["visible"][3*match_index+1] = True # Predicted trace
    button_dict["args"][0]["visible"][3*match_index+2] = True # Goal trace
    
    # set visibility to true for the corresponding traces for the first match
    if match_index == 0:
        button_dict["args"][0]["visible"][0] = True # Bookmaker odds trace
        button_dict["args"][0]["visible"][1] = True # Predicted trace
        button_dict["args"][0]["visible"][2] = True
        
    # append the button to the list
    buttons.append(button_dict)


# update the layout with the buttons
fig.update_layout(
    updatemenus=[go.layout.Updatemenu(
        buttons=buttons,
        active=0, # set the initial active button
        x=0.1,
        y=1.2,
        direction="down"
    )],
    
)

# show the plot
fig.show()


In [ ]:
Pont = 5
bank = 50
Final = Final_Odds[Final_Odds['Type']=='Predicted'][['Match']]
listPred = Final_Odds[Final_Odds['Type']=='Predicted']['P(Under)'].tolist()
listBook = Final_Odds[Final_Odds['Type']=='Bookmaker']['P(Under)'].tolist()

diff = []
for i, p in enumerate(listPred):
    diff.append(listPred[i]-listBook[i])

Final['UnderBet'] = np.nan
Final['UnderBet_Kelly'] = np.nan
for i, d in enumerate(diff):
    if (0.7>=listPred[i]>=0.35) and (d>=0.001):
        Final['UnderBet'][i] = max(Pont*10*(d),0)
        Final['UnderBet_Kelly'][i] = max(0, bank*(listPred[i]- (1-listPred[i])/(1/listBook[i] -1)))
    else:
        Final['UnderBet'][i] = 0
        Final['UnderBet_Kelly'][i] = 0
        
############

    
display(Final)

/tmp/ipykernel_147206/2079959016.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_147206/2079959016.py:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_147206/2079959016.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_147206/2079959016.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pa

,Match,UnderBet,UnderBet_Kelly
0,Bournemouth-Chelsea,0.000,0.000
1,Manchester City-Leeds,2.984,4.254
2,Tottenham-Crystal Palace,0.000,0.000
3,Wolverhampton Wanderers-Aston Villa,0.000,0.000
4,Liverpool-Brentford,0.000,0.000
5,Newcastle United-Arsenal,0.000,0.000
6,West Ham-Manchester United,0.000,0.000
7,Fulham-Leicester,3.099,5.995
8,Brighton-Everton,0.774,1.347
9,Nottingham Forest-Southampton,0.000,0.000
